# Knight's tour

A knight's tour is a sequence of moves of a knight on a chessboard such that the knight visits every square exactly once. If the knight ends on a square that is one knight's move from the beginning square (so that it could tour the board again immediately, following the same path), the tour is closed; otherwise, it is open.

ref: https://en.wikipedia.org/wiki/Knight%27s_tour

In [1]:
!git clone https://github.com/maxtuno/SLIME

Cloning into 'SLIME'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (323/323), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 323 (delta 142), reused 271 (delta 95), pack-reused 0
Receiving objects: 100% (323/323), 673.22 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [2]:
cd /content/SLIME/SLIME

/content/SLIME/SLIME


In [3]:
mkdir build

In [4]:
cd build

/content/SLIME/SLIME/build


In [5]:
!cmake -D CMAKE_BUILD_TYPE=Release ..

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found MPI_C: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so (found version "3.1") 
-- Found MPI_CXX: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi_cxx.so (found version "3.1") 
-- Found MPI: TRUE (found version "3.1")  
-- Configuring done
-- Generating done
-- Build files have been written to: /content/SLIME/SLIME/build


In [6]:
!make

Scanning dependencies of target slime
[ 16%] Building CXX object CMakeFiles/slime.dir/src/Ls.o
[ 33%] Building CXX object CMakeFiles/slime.dir/src/Main.o
[ 50%] Building CXX object CMakeFiles/slime.dir/src/Options.o
[ 66%] Building CXX object CMakeFiles/slime.dir/src/SimpSolver.o
[ 83%] Building CXX object CMakeFiles/slime.dir/src/Solver.o
[100%] Linking CXX executable ../bin/slime
[100%] Built target slime


In [7]:
!pip install peqnp
!pip install numpy

     |████████████████████████████████| 81kB 4.7MB/s 
  Created wheel for peqnp: filename=PEQNP-6.0.3-cp37-cp37m-linux_x86_64.whl size=729738 sha256=1c03daf86d32dbcb52e085fbfcd0ee36b926d3978cc0f70fc9b0fed67307132c
  Stored in directory: /root/.cache/pip/wheels/9c/97/71/ea4c2c381b6ac91820551d6029bbab4e766ff1a4e894d04c2c
Successfully built peqnp


In [8]:
%%time
import numpy as np
import peqnp as pn

n = 6
m = n ** 2

pn.engine(m.bit_length())

# note: use key parameter for variables to capture.
r = pn.vector(size=m, key='r')
c = pn.vector(size=m, key='c')
rr = pn.vector(size=m)
cc = pn.vector(size=m)

pn.apply_single(r + c, lambda x: 1 <= x <= n)
pn.apply_single(rr + cc, lambda x: x.is_in([1, 2]))
pn.all_different([r[i] + n * c[i] for i in range(m)])

for i in range(m - 1):
    assert c[i + 1] == pn.one_of([c[i] + cc[i], c[i] - cc[i]])
    assert r[i + 1] == pn.one_of([r[i] + rr[i], r[i] - rr[i]])
    assert cc[i] != rr[i]

if pn.external_satisfy(key='knight', solver='mpirun -q -np 4 --allow-run-as-root /content/SLIME/SLIME/bin/slime'):
    c = np.vectorize(int)(c) - 1
    r = np.vectorize(int)(r) - 1
    t = np.zeros(shape=(n, n), dtype=int)
    for k, (i, j) in enumerate(zip(c, r)):
        t[i][j] = k + 1
    print(t)
else:
    print('Infeasible ...')


[[20 33  4 13 22 31]
 [ 3 12 21 32  5 14]
 [34 19  6 15 30 23]
 [11  2 35 26  7 16]
 [36 27 18  9 24 29]
 [ 1 10 25 28 17  8]]
CPU times: user 423 ms, sys: 11.2 ms, total: 434 ms
Wall time: 42.7 s
